<div style="background-color:grey; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">

### Lab 5: Orchestrating Agents with MemGPT

</div>

### 1. Setup and Configuration
This first block handles the initial setup, including 
- installing the library and 
- configuring your Gemini API key using the helper.py file.

In [1]:
# Install the Google Generative AI library
#!pip install -q google-generativeai

import google.generativeai as genai
import os
import sys
import uuid
import json
import time
from IPython.display import display, Markdown
# Import the function from your helper file
from helper import get_gemini_api_key

def print_markdown(text):
    """Prints text as markdown in a notebook."""
    display(Markdown(text))

# --- Configuration ---
try:
    api_key = get_gemini_api_key()
    if not api_key:
        raise ValueError("API key is missing. Please ensure your helper.py file returns a valid key.")
    genai.configure(api_key=api_key)
    print("\nGemini API configured successfully!")
except Exception as e:
    print(f"An error occurred during configuration: {e}")

# --- Resume Files ---
# Create dummy resume files for the script to read
with open("tony_stark.txt", "w") as f:
    f.write("Tony Stark: Genius, billionaire, playboy, philanthropist. Expert in robotics and AI. Built the Iron Man suit. Strong software engineering skills.")

with open("spongebob_squarepants.txt", "w") as f:
    f.write("Spongebob Squarepants: Fry cook at the Krusty Krab. Enthusiastic and dedicated, but lacks technical skills in software engineering.")



Gemini API configured successfully!


/opt/anaconda3/envs/gl_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2. Core Agent and Orchestration Logic
This section contains 
- the reusable GeminiAgent class and 
- the new run_orchestration function, which will manage the interactions between our different agents.

In [2]:
# --- Agent Class ---
class GeminiAgent:
    """A class to simulate an agent's state and memory."""
    def __init__(self, name, model_name, system_prompt="", memory_blocks=None, tools=None):
        self.id = f"agent-{uuid.uuid4()}"
        self.name = name
        self.memory_blocks = memory_blocks if memory_blocks else {}
        self.tools = tools if tools else []
        self.system_prompt = system_prompt
        self.model = genai.GenerativeModel(
            model_name=model_name,
            tools=self.tools
        )

    def get_formatted_memory(self):
        """Formats memory blocks into a string for the system prompt."""
        if not self.memory_blocks:
            return ""
        formatted_string = "--- CORE MEMORY ---\n"
        for label, value in self.memory_blocks.items():
            val_str = json.dumps(value) if isinstance(value, list) else str(value)
            formatted_string += f"<{label}>\n{val_str}\n</{label}>\n"
        return formatted_string.strip()

# --- Tool Definitions (for the model) ---
def draft_candidate_email(content: str):
    """
    Draft an email to reach out to a candidate.
    Args:
        content (str): Content of the email.
    """
    pass

def reject(candidate_name: str):
    """
    Reject a candidate.
    Args:
        candidate_name (str): The name of the candidate.
    """
    pass

# --- Tool Implementations (for Python) ---
def _draft_candidate_email_impl(content: str):
    """Implementation for drafting an email."""
    print(f"📧 **Email Drafted:**\n---\n{content}\n---\n")
    return f"Email draft completed."

def _reject_impl(candidate_name: str):
    """Implementation for rejecting a candidate."""
    print(f"🚫 **Candidate Rejected:** {candidate_name}\n")
    return f"Candidate {candidate_name} has been rejected."

# --- Orchestration Logic ---
def run_orchestration(initial_agent, agents_map, tool_registries, initial_message):
    """Manages a multi-agent workflow."""
    print_markdown(f"### 👤 User: {initial_message}")
    print("---")

    current_agent = initial_agent
    message_to_agent = initial_message
    history = []

    for i in range(5): # Limit loops to prevent infinite cycles
        print_markdown(f"#### ➡️ Turn {i+1}: Passing control to `{current_agent.name}`")
        
        full_prompt = f"{current_agent.system_prompt}\n\n{current_agent.get_formatted_memory()}\n\n**New Task:**\n{message_to_agent}"
        history.append({'role': 'user', 'parts': [{'text': full_prompt}]})

        response = current_agent.model.generate_content(history, tools=current_agent.tools)
        time.sleep(1)

        # Safety check for empty or blocked responses
        if not response.candidates:
            print_markdown(f"### 🤖 Agent ({current_agent.name}): No response was generated. This might be due to safety settings.")
            print_markdown("#### ✅ Orchestration Complete")
            return

        message = response.candidates[0].content
        history.append(message)

        # Handle tool calls
        if any(part.function_call for part in message.parts):
            tool_response_parts = []
            for part in message.parts:
                if not part.function_call: continue
                fc = part.function_call
                tool_name = fc.name
                tool_args = dict(fc.args)
                print(f"🧠 **Reasoning ({current_agent.name}):** Model wants to call `{tool_name}`.")
                print(f"🔧 **Tool Call ({current_agent.name}):** `{tool_name}` with arguments: `{tool_args}`\n" + "---")
                
                result = tool_registries[current_agent.name][tool_name](**tool_args)
                tool_response_parts.append({"function_response": {"name": tool_name, "response": {"content": result}}})
            
            history.append({"role": "tool", "parts": tool_response_parts})
            response = current_agent.model.generate_content(history, tools=current_agent.tools)
            time.sleep(1)
            
            if not response.candidates:
                print_markdown(f"### 🤖 Agent ({current_agent.name}): No response was generated after a tool call. This might be due to safety settings.")
                print_markdown("#### ✅ Orchestration Complete")
                return

            message = response.candidates[0].content
            history.append(message)

        # Safely access the final text
        final_text = message.parts[0].text if message.parts else "The agent did not provide a text response."
        print_markdown(f"### 🤖 Agent ({current_agent.name}): {final_text}")

        # Check for handoff instructions
        if "NEXT_AGENT:" in final_text:
            parts = final_text.split("MESSAGE:", 1)
            next_agent_name = parts[0].replace("NEXT_AGENT:", "").strip()
            message_to_agent = parts[1].strip() if len(parts) > 1 else ""
            current_agent = agents_map[next_agent_name]
        else:
            print_markdown("#### ✅ Orchestration Complete")
            break


### GENERATE - Sample Resumes

In [3]:
# Install the Google Generative AI library
#!pip install -q google-generativeai

import google.generativeai as genai
import os
import sys
import uuid
import json
import time
from IPython.display import display, Markdown
# Import the function from your helper file
from helper import get_gemini_api_key

def print_markdown(text):
    """Prints text as markdown in a notebook."""
    display(Markdown(text))

# --- Configuration ---
try:
    api_key = get_gemini_api_key()
    if not api_key:
        raise ValueError("API key is missing. Please ensure your helper.py file returns a valid key.")
    genai.configure(api_key=api_key)
    print("\nGemini API configured successfully!")
except Exception as e:
    print(f"An error occurred during configuration: {e}")

# --- Resume Files ---
# Create dummy resume files for the script to read
with open("tony_stark.txt", "w") as f:
    f.write("Tony Stark: Genius, billionaire, playboy, philanthropist. Expert in robotics and AI. Built the Iron Man suit. Strong software engineering skills.")

with open("spongebob_squarepants.txt", "w") as f:
    f.write("Spongebob Squarepants: Fry cook at the Krusty Krab. Enthusiastic and dedicated, but lacks technical skills in software engineering.")



Gemini API configured successfully!


### 3. Multi-Agent Orchestration
This section 
- sets up the two agents (eval_agent and outreach_agent), 
- defines their shared memory, and 
- runs the orchestration to evaluate a candidate's resume.

In [4]:
# --- 1. Shared Memory ---
# This dictionary will be passed to both agents to simulate a shared memory block.
shared_memory = {
    "company": "The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents."
}

# --- 2. Create Outreach Agent ---
outreach_persona = (
    "You are responsible for drafting emails on behalf of a company using the `draft_candidate_email` tool. "
    "You will receive instructions to draft emails for specific candidates."
)
outreach_agent = GeminiAgent(
    name="outreach_agent",
    model_name="gemini-1.5-flash",
    system_prompt=outreach_persona,
    memory_blocks=shared_memory,
    tools=[draft_candidate_email]
)

# --- 3. Create Evaluation Agent ---
skills = "Front-end (React, Typescript) or software engineering skills"
eval_persona = (
    f"You are responsible for evaluating candidates based on their resumes. Ideal candidates have skills in: {skills}. "
    "You have two choices: \n"
    "1. If the candidate is a bad fit, use your `reject` tool. \n"
    "2. If the candidate is a strong fit, you MUST hand them off to the `outreach_agent`. To do this, respond with the exact format: 'NEXT_AGENT: outreach_agent MESSAGE: [Your message to the outreach agent, e.g., 'Draft an email for candidate X']'"
)
eval_agent = GeminiAgent(
    name="eval_agent",
    model_name="gemini-1.5-flash",
    system_prompt=eval_persona,
    memory_blocks=shared_memory,
    tools=[reject]
)

# --- 4. Setup Registries and Agent Map ---
agents_map = {
    "eval_agent": eval_agent,
    "outreach_agent": outreach_agent
}
tool_registries = {
    "eval_agent": {"reject": _reject_impl},
    "outreach_agent": {"draft_candidate_email": _draft_candidate_email_impl}
}

# --- 5. Run the Orchestration ---
resume = open("tony_stark.txt", "r").read()
run_orchestration(
    initial_agent=eval_agent,
    agents_map=agents_map,
    tool_registries=tool_registries,
    initial_message=f"Evaluate this resume: {resume}"
)

# --- 6. Demonstrate Shared Memory Update ---
print_markdown("\n--- \n### Demonstrating Shared Memory Update")
print(f"Original company name in eval_agent's memory: '{eval_agent.memory_blocks['company']}'")
shared_memory['company'] = "The company has rebranded to Letta"
print(f"Updated company name in eval_agent's memory: '{eval_agent.memory_blocks['company']}'")


### 👤 User: Evaluate this resume: Tony Stark: Genius, billionaire, playboy, philanthropist. Expert in robotics and AI. Built the Iron Man suit. Strong software engineering skills.

---


#### ➡️ Turn 1: Passing control to `eval_agent`

### 🤖 Agent (eval_agent): The agent did not provide a text response.

#### ✅ Orchestration Complete


--- 
### Demonstrating Shared Memory Update

Original company name in eval_agent's memory: 'The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.'
Updated company name in eval_agent's memory: 'The company has rebranded to Letta'


### 4. Simulating Agent Groups (Round-Robin)
This final section simulates 
- the "agent group" concept by creating a simple round-robin orchestrator that passes a message from one agent to the next in a predefined sequence.

In [5]:
def run_group_orchestration(agents, tool_registries, initial_message):
    """Simulates a round-robin agent group."""
    print_markdown(f"### 👤 User: {initial_message}")
    print("---")
    
    message_for_next_agent = initial_message
    history = []

    for i, agent in enumerate(agents):
        print_markdown(f"#### ➡️ Turn {i+1}: Passing control to `{agent.name}` (Round-Robin)")
        
        full_prompt = f"{agent.system_prompt}\n\n{agent.get_formatted_memory()}\n\n**New Task:**\n{message_for_next_agent}"
        history.append({'role': 'user', 'parts': [{'text': full_prompt}]})
        
        response = agent.model.generate_content(history, tools=agent.tools)
        time.sleep(1)
        message = response.candidates[0].content
        history.append(message)

        if any(part.function_call for part in message.parts):
            tool_response_parts = []
            for part in message.parts:
                if not part.function_call: continue
                fc = part.function_call
                tool_name = fc.name
                tool_args = dict(fc.args)
                print(f"🔧 **Tool Call ({agent.name}):** `{tool_name}` with arguments: `{tool_args}`\n" + "---")
                result = tool_registries[agent.name][tool_name](**tool_args)
                tool_response_parts.append({"function_response": {"name": tool_name, "response": {"content": result}}})
            
            history.append({"role": "tool", "parts": tool_response_parts})
            response = agent.model.generate_content(history, tools=agent.tools)
            time.sleep(1)
            message = response.candidates[0].content
            history.append(message)

        final_text = message.parts[0].text
        print_markdown(f"### 🤖 Agent ({agent.name}): {final_text}")
        
        # The output of one agent becomes the input for the next
        message_for_next_agent = final_text
        
    print_markdown("#### ✅ Group Orchestration Complete")

# --- Run the Group Simulation ---
resume = open("spongebob_squarepants.txt", "r").read()
# The group is simply a list of the agents in the desired order
agent_group = [eval_agent, outreach_agent] 

run_group_orchestration(
    agents=agent_group,
    tool_registries=tool_registries,
    initial_message=f"Evaluate this resume: {resume}"
)


### 👤 User: Evaluate this resume: Spongebob Squarepants: Fry cook at the Krusty Krab. Enthusiastic and dedicated, but lacks technical skills in software engineering.

---


#### ➡️ Turn 1: Passing control to `eval_agent` (Round-Robin)

🔧 **Tool Call (eval_agent):** `reject` with arguments: `{'candidate_name': 'Spongebob Squarepants'}`
---
🚫 **Candidate Rejected:** Spongebob Squarepants



### 🤖 Agent (eval_agent): OK. Spongebob Squarepants was rejected because he lacks the required technical skills.


#### ➡️ Turn 2: Passing control to `outreach_agent` (Round-Robin)

### 🤖 Agent (outreach_agent): OK.  I understand.  I'm ready for the next candidate's information.


#### ✅ Group Orchestration Complete